In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO



In [10]:
import json
f = open('compare_AB.json', 'r', encoding='UTF-8-sig')

compare_ab = json.load(f)

In [4]:
f = open('compare_AC.json', 'r', encoding='UTF-8-sig')

compare_ac = json.load(f)

In [5]:
f = open('compare_AD.json', 'r', encoding='UTF-8-sig')

compare_ad = json.load(f)

In [81]:
compare_ad['195699'] # -8.8

{'니치향수': [6, 12, -6],
 '라임바질': [0, 0, 0],
 '라임바질앤만다린': [0, 0, 0],
 '조말론': [14, 15, -1],
 '향수': [68, 105, -37]}

In [82]:
compare_ad['206825'] # -9.9

{'18k': [96, 112, -16],
 '18k 금팔찌': [18, 26, -8],
 '24k': [46, 58, -12],
 '24k금팔찌': [10, 12, -2],
 '구찌팔찌': [0, 3, -3],
 '금목걸이': [44, 57, -13],
 '금팔찌': [40, 52, -12],
 '남자금팔찌': [27, 39, -12],
 '남자팔찌': [33, 36, -3],
 '대나무목걸이': [19, 29, -10],
 '도금 ': [44, 57, -13],
 '도금목걸이': [42, 55, -13],
 '도금반지': [42, 55, -13],
 '도금팔찌': [43, 57, -14],
 '순금': [46, 59, -13],
 '순금목걸이': [30, 42, -12],
 '순금팔찌': [32, 40, -8],
 '체인목걸이': [19, 27, -8],
 '체인팔찌 ': [14, 19, -5],
 '초굴빵': [16, 23, -7],
 '팔각팔찌': [26, 37, -11]}

In [83]:
compare_ad['195726'] # -9.16

{'네롤리': [0, 0, 0],
 '네롤리포르토피노': [0, 0, 0],
 '니치향수': [30, 48, -18],
 '톰포드': [5, 11, -6],
 '톰포드향수': [8, 14, -6],
 '향수': [76, 101, -25]}

In [84]:
compare_ad['195770'] # -11

{'끌로에': [1, 1, 0],
 '끌로에향수': [0, 0, 0],
 '비누향': [1, 1, 0],
 '여자향수': [19, 41, -22],
 '향수': [27, 60, -33]}

In [85]:
compare_ad['195681'] # -25.6

{'니치향수': [4, 54, -50],
 '바이레도': [2, 4, -2],
 '바이레도향수': [0, 0, 0],
 '집시워터': [0, 0, 0],
 '향수': [31, 107, -76]}

In [48]:
def count_less_than(compare_json, diff):
    result = {
        'ad_set_product_id': [],
        'bid_keyword_cnt': [],
        'over_bid_keyword_cnt': []
    }
    for product_id, bids in compare_json.items():
        bid_keyword_cnt = 0
        over_bid_keyword_cnt = 0
        for bid_keyword, rank in bids.items():
            bid_keyword_cnt += 1
            if rank[2] is not None:
                if rank[2] <= diff:
                    over_bid_keyword_cnt += 1
        ad_set_product_id_list = result.get('ad_set_product_id', [])
        bid_keyword_cnt_list = result.get('bid_keyword_cnt', [])
        over_bid_keyword_cnt_list = result.get('over_bid_keyword_cnt', [])
        ad_set_product_id_list.append(product_id)
        bid_keyword_cnt_list.append(bid_keyword_cnt)
        over_bid_keyword_cnt_list.append(over_bid_keyword_cnt)
        result['ad_set_product_id'] = ad_set_product_id_list
        result['bid_keyword_cnt'] = bid_keyword_cnt_list
        result['over_bid_keyword_cnt'] = over_bid_keyword_cnt_list
    
    return result
                

In [52]:
ab_ = count_less_than(compare_ab, -12)
ab_12 = pd.DataFrame.from_dict(ab_)

In [54]:
ab_12[ab_12['over_bid_keyword_cnt'] > 0].shape

(5, 3)

In [55]:
ac_ = count_less_than(compare_ac, -12)
ac_12 = pd.DataFrame.from_dict(ac_)
ac_12[ac_12['over_bid_keyword_cnt'] > 0].shape

(13, 3)

In [56]:
ad_ = count_less_than(compare_ad, -12)
ad_12 = pd.DataFrame.from_dict(ad_)
ad_12[ad_12['over_bid_keyword_cnt'] > 0].shape

(32, 3)

In [23]:
from statistics import mean
def get_mean_drop_rank(compare_json):
    result = {
        'ad_product_id': [],
        'bid_keyword_cnt': [],
        'mean_drop_rank': []
    }
    for product_id, bids in compare_json.items():
        drop_ranks = []
        cnt = 0
        for bid_keyword, rank in bids.items():
            if rank[2] is not None:
                cnt += 1
                drop_ranks.append(rank[2])
        ad_product_id_list = result.get('ad_product_id', [])
        bid_keyword_cnt_list = result.get('bid_keyword_cnt', [])
        mean_drop_rank_list = result.get('mean_drop_rank', [])
        ad_product_id_list.append(product_id)
        bid_keyword_cnt_list.append(cnt)
        mean_drop_rank_list.append(mean(drop_ranks))
        result['ad_product_id'] = ad_product_id_list
        result['bid_keyword_cnt'] = bid_keyword_cnt_list
        result['mean_drop_rank'] = mean_drop_rank_list
    return result

In [24]:
ab = get_mean_drop_rank(compare_ab)

In [30]:
pd.DataFrame.from_dict(ab).describe()

,bid_keyword_cnt,mean_drop_rank
count,175.000000,175.000000
mean,6.571429,0.008123
std,4.431378,1.498566
min,1.000000,-6.625000
25%,5.000000,-0.183333
50%,5.000000,0.000000
75%,6.000000,0.400000
max,24.000000,10.150000


In [41]:
ab2 = pd.DataFrame.from_dict(ab)
ab2[ab2['mean_drop_rank'] <= -6]

,ad_product_id,bid_keyword_cnt,mean_drop_rank
50,195683,8,-6.625
137,195769,5,-6.000


In [67]:
def get_df(df):
    tmp1 = df.groupby('bid_keyword_cnt')['mean_drop_rank'].apply(list).reset_index()
    tmp2 = df.groupby('bid_keyword_cnt')['mean_drop_rank'].count().reset_index()
    tmp3 = df.groupby('bid_keyword_cnt')['mean_drop_rank'].mean().reset_index()
    tmp1.columns = ['bid_keyword_cnt', 'mean_drop_rank_list']
    tmp2.columns = ['bid_keyword_cnt', 'cnt']
    tmp3.columns = ['bid_keyword_cnt', 'average']
    result = pd.merge(tmp1, tmp2, on='bid_keyword_cnt')
    result = pd.merge(result, tmp3, on='bid_keyword_cnt')
    return result

In [71]:
get_df(ab2).to_csv('ab2.csv', index=False)
get_df(ab2)

,bid_keyword_cnt,mean_drop_rank_list,cnt,average
0,1,"[0.0, 0.0, 0.0, 0.0, 1.0]",5,0.200000
1,2,"[0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 3.5, 0.0, -1.0...",13,0.192308
2,3,"[0.0, 1.0, -3.3333333333333335]",3,-0.777778
3,4,"[0.0, 0.0, 0.0, -1.0, 0.0, 0.0, -3.25, 0.0, 0....",10,-0.425000
4,5,"[0.2, 0.4, 0.6, 0.0, -0.2, 0.4, -0.4, 1.0, -0....",62,-0.080645
5,6,"[0.0, -1.1666666666666667, 0.3333333333333333,...",39,0.115385
6,7,"[0.0, -1.2857142857142858, 0.0, -1.0, 0.142857...",12,-0.107143
7,8,"[1.125, -0.125, -6.625, 0.0, -0.125, 0.375, -0...",7,-0.821429
8,9,"[-0.4444444444444444, 0.4444444444444444, -0.6...",4,-0.305556
9,10,"[-0.1, 0.1]",2,0.000000


In [72]:
get_df(ac2).to_csv('ac2.csv', index=False)
get_df(ac2)

,bid_keyword_cnt,mean_drop_rank_list,cnt,average
0,1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",11,0.090909
1,2,"[-1.0, -0.5, 1.0, 2.0, 0.0, 1.5, 0.0, -1.0]",8,0.250000
2,3,"[0.0, 0.0, 0.6666666666666666, 1.0, 0.0, 0.0, ...",10,-0.033333
3,4,"[0.0, 0.0, 0.0, 0.0, 0.75, 0.25, -2.5, -0.5, 0...",19,-0.355263
4,5,"[-0.4, -1.8, -1.2, -1.2, -5.6, 0.0, 28.4, -1.8...",66,-0.148485
5,6,"[0.16666666666666666, 0.8333333333333334, 0.16...",38,0.627193
6,7,"[0.0, -0.8571428571428571, -1.4285714285714286...",15,0.190476
7,8,"[0.25, -0.5, -6.75, -0.125, -1.625, -0.5, -0.75]",7,-1.428571
8,9,"[-0.5555555555555556, 2.111111111111111, 0.111...",6,0.148148
9,10,"[3.2, 0.1, 1.1, 1.1, 0.0]",5,1.100000


In [73]:
get_df(ad2).to_csv('ad2.csv', index=False)
get_df(ad2)

,bid_keyword_cnt,mean_drop_rank_list,cnt,average
0,1,"[0.0, 0.0]",2,0.000000
1,2,"[0.5, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0]",7,-0.071429
2,3,"[0.0, 1.0, 0.0, 1.3333333333333333, 0.0, -3.66...",9,0.074074
3,4,"[0.0, 0.0, 0.0, -1.25, 0.25, -3.0, -0.75, 0.0,...",18,-0.347222
4,5,"[-3.2, -2.8, -6.4, 17.2, -1.8, 27.4, -2.0, -0....",64,-0.781250
5,6,"[-1.0, 0.3333333333333333, 0.6666666666666666,...",39,-0.769231
6,7,"[0.42857142857142855, -2.4285714285714284, -2....",13,-0.604396
7,8,"[1.0, -1.0, -6.25, -0.125, -1.875, -0.875, -0.75]",7,-1.410714
8,9,"[-1.6666666666666667, 1.8888888888888888, -1.4...",6,-0.629630
9,10,"[1.9, 0.1, 1.2, 1.0, 0.0]",5,0.840000


In [31]:
ac = get_mean_drop_rank(compare_ac)

In [32]:
pd.DataFrame.from_dict(ac).describe()

,bid_keyword_cnt,mean_drop_rank
count,208.000000,208.000000
mean,6.519231,0.082147
std,4.417752,4.141787
min,1.000000,-24.600000
25%,5.000000,-0.648810
50%,5.000000,0.000000
75%,7.000000,0.833333
max,24.000000,28.600000


In [42]:
ac2 = pd.DataFrame.from_dict(ac)
ac2[ac2['mean_drop_rank'] <= -6]

,ad_product_id,bid_keyword_cnt,mean_drop_rank
38,195687,5,-7.00
43,195694,5,-6.60
60,195683,8,-6.75
123,195707,5,-6.80
133,195681,5,-24.60
161,195769,5,-6.80


In [33]:
ad = get_mean_drop_rank(compare_ad)
pd.DataFrame.from_dict(ad).describe()

,bid_keyword_cnt,mean_drop_rank
count,240.000000,240.000000
mean,9.841667,-0.338062
std,7.321083,5.137093
min,1.000000,-25.600000
25%,5.000000,-1.931818
50%,6.000000,-0.093750
75%,17.250000,0.600000
max,26.000000,27.400000


In [43]:
ad2 = pd.DataFrame.from_dict(ad)
ad2[ad2['mean_drop_rank'] <= -6]


,ad_product_id,bid_keyword_cnt,mean_drop_rank
17,195778,6,-6.166667
21,195738,5,-6.400000
26,195684,6,-7.166667
41,195687,5,-8.000000
60,195695,5,-7.000000
68,195683,8,-6.250000
73,206812,22,-6.590909
82,206843,23,-7.130435
102,195749,5,-7.000000
116,195711,5,-7.800000
